# Criação do modelo de vetorização de cursos

## Importando os módulos necessários

In [12]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

## Usando uma build pré-treinada e iniciando modelo

In [13]:
encoder_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
encoder = hub.KerasLayer(encoder_url, input_shape=[], dtype=tf.string, trainable=True)

## Importando os dados para vetorização e criação do espaço vetorial

+ Os dados devem estar armazenados no mesmo diretório que o código
+ Serão importados na forma de Dataframe, usando Pandas
+ Após importação, é feita a concatenação das variáveis "Course Name", "Course Description" e "all_skill", pois juntas formarão o texto usado para produzir a representação vetorial do curso.
+ Os campos são então armazenados na forma de listas, uma para cada coluna do Dataframe

In [18]:
df = pd.read_csv("dataset.csv")

df['Concatenation'] = df['regulation'].astype(str)


list = df.values.tolist()

names = []
num = []
paragraph = []
sentences = []
for i in range(len(list)):
    sentences.append(list[i][3])
    names.append(list[i][0])
    num.append(list[i][1])
    paragraph.append(list[i][2])

df.head(1)

,dataset,regulation,num,paragraph,Concatenation
0,Quais as modalidades de cursos de graduacao de...,Os cursos de graduação da Universidade Federal...,Art. 1,NaN,Os cursos de graduação da Universidade Federal...


## Criando o modelo

+ A camada de input receberá um texto
+ Este texto é processado pelas camadas do Universal Language encoder, modelo extraído do Tensorflow Hub
+ O modelo é então criado com a arquitetura definida
+ O modelo criado é compilado e salvo.

In [19]:
input1 = tf.keras.Input(shape=[], dtype=tf.string)
embedding_layer = encoder(input1)

finishedModel = tf.keras.Model(inputs=input1, outputs=embedding_layer)
finishedModel.compile(optimizer="adam", loss="binary_crossentropy", metrics=["AUC"])
finishedModel.save("searchModel.h5", include_optimizer=False)

2023-06-12 16:48:13.688333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]


## Criando vetores

+ Para criar-se os vetores, o modelo é utilizado para codificar todos os itens da lista sentence
+ Esta lista, criada anteriormente, possui a concatenação do título, descrição e habilidades de cada curso, na mesma ordem em que aparecem no dataset.

In [20]:
vectors = finishedModel.predict(num)
print(num)


7/7 [==============================] - 4s 203ms/step


## Criando espaço vetorial
+ Os vetores criados anteriormente devem ser persistidos em JSON
+ O JSON possui diversos campos, cada um contendo um dado relevante acerca de um curso.
+ Para cada vetor armazenado no campo "data", existe um metadado salvo em cada um dos outros campos no mesmo índice
+ Verificando-se um dado índice em todos os campos, é possível recuperar todas as informações de um curso.

In [21]:
import json
lists = vectors.tolist()
data = {"data": lists,
        "names": names,
        "num": num,
        "paragraph": paragraph
       }
json_str = json.dumps(data)

f = open('data.json', 'w')
f.write(json_str)
f.close()
